# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [3]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

trans = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rew = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

POLICY_ACTION = "do-nothing"


states = sorted(set(trans["state"]).union(set(trans["next_state"])))

trans_pi = trans[trans["action"] == POLICY_ACTION].copy()
rew_pi = rew[rew["action"] == POLICY_ACTION].copy()

R = {row["state"]: float(row["reward"]) for _, row in rew_pi.iterrows()}
for s in states:
    R.setdefault(s, 0.0)

P = {s: [] for s in states}
for _, row in trans_pi.iterrows():
    s = row["state"]
    sp = row["next_state"]
    p = float(row["probability"])
    P[s].append((sp, p))

tol = 1e-12
max_iters = 200000

V = {s: 0.0 for s in states}

for _ in range(max_iters):
    delta = 0.0
    V_new = {}

    for s in states:
        expected_next = sum(p * V[sp] for sp, p in P.get(s, []))
        v = R[s] + expected_next
        V_new[s] = v
        delta = max(delta, abs(v - V[s]))

    V = V_new
    if delta < tol:
        break

discomfort = {s: -V[s] for s in states}

Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

In [2]:
# YOUR CHANGES HERE

out_df = pd.DataFrame([
    {"state": s, "expected_discomfort": discomfort[s]}
    for s in sorted(discomfort.keys())
])

out_df.to_csv("do-nothing-discomfort.tsv", sep="\t", index=False)

out_df

,state,expected_discomfort
0,exposed-1,3.413333
1,exposed-2,4.266667
2,exposed-3,5.333333
3,recovered,-0.000000
4,symptoms-1,6.666667
5,symptoms-2,5.000000
6,symptoms-3,1.666667


Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [4]:
# YOUR CHANGES HERE


#trans = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
#rew = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

states = sorted(set(trans["state"]).union(set(trans["next_state"])))

R = {}
for _, row in rew.iterrows():
    R[(row["state"], row["action"])] = float(row["reward"])

P = {}
for _, row in trans.iterrows():
    key = (row["state"], row["action"])
    P.setdefault(key, []).append((row["next_state"], float(row["probability"])))

actions_by_state = {s: set() for s in states}
for (s, a) in P.keys():
    actions_by_state[s].add(a)
for (s, a) in R.keys():
    actions_by_state.setdefault(s, set()).add(a)

tol = 1e-12
max_iters = 200000

V = {s: 0.0 for s in states}

for _ in range(max_iters):
    delta = 0.0
    V_new = {}
    for s in states:
        actions = list(actions_by_state.get(s, []))
        if not actions:
            V_new[s] = 0.0
            continue
        best = -float("inf")
        for a in actions:
            r = R.get((s, a), 0.0)
            exp_next = 0.0
            for sp, p in P.get((s, a), []):
                exp_next += p * V[sp]
            q = r + exp_next
            if q > best:
                best = q
        V_new[s] = best
        delta = max(delta, abs(V_new[s] - V[s]))
    V = V_new
    if delta < tol:
        break

policy = {}
for s in states:
    actions = list(actions_by_state.get(s, []))
    if not actions:
        policy[s] = ""
        continue
    best_a = None
    best_q = -float("inf")
    for a in actions:
        r = R.get((s, a), 0.0)
        exp_next = sum(p * V[sp] for sp, p in P.get((s, a), []))
        q = r + exp_next
        if q > best_q:
            best_q = q
            best_a = a
    policy[s] = best_a

Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [5]:
# YOUR CHANGES HERE

out_df = pd.DataFrame([{"state": s, "action": policy[s]} for s in states])
out_df.to_csv("minimum-discomfort-actions.tsv", sep="\t", index=False)

out_df

,state,action
0,exposed-1,sleep-8
1,exposed-2,sleep-8
2,exposed-3,sleep-8
3,recovered,sleep-8
4,symptoms-1,drink-oj
5,symptoms-2,drink-oj
6,symptoms-3,drink-oj


Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [ ]:
# YOUR CHANGES HERE

T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
R = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")
Pi = pd.read_csv("minimum-discomfort-actions.tsv", sep="\t")

T.columns = [c.strip().lower() for c in T.columns]
R.columns = [c.strip().lower() for c in R.columns]
Pi.columns = [c.strip().lower() for c in Pi.columns]

def pick(col_options, cols):
    for c in col_options:
        if c in cols:
            return c
    raise ValueError(f"Missing one of {col_options}; found columns={list(cols)}")

s_col  = pick(["state", "s"], T.columns)
a_col  = pick(["action", "a"], T.columns)
sp_col = pick(["next_state", "next", "sprime", "s_next", "sp"], T.columns)
p_col  = pick(["prob", "probability", "p"], T.columns)

T = T[[s_col, a_col, sp_col, p_col]].copy()
T[s_col] = T[s_col].astype(str)
T[a_col] = T[a_col].astype(str)
T[sp_col] = T[sp_col].astype(str)
T[p_col] = T[p_col].astype(float)

states = sorted(set(T[s_col]).union(set(T[sp_col])))

Pi_map = {str(r["state"]): str(r["action"]) for _, r in Pi.iterrows()}

if "reward" not in R.columns:
    raise ValueError(f"Rewards file must have a 'reward' column; found columns={list(R.columns)}")

if "next_state" not in R.columns:
    for alt in ["next", "sprime", "s_next", "sp"]:
        if alt in R.columns:
            R = R.rename(columns={alt: "next_state"})
            break

r_sas = None
r_sa = None
r_s = None

if ("state" in R.columns) and ("action" in R.columns) and ("next_state" in R.columns):
    r_sas = {(str(row["state"]), str(row["action"]), str(row["next_state"])): float(row["reward"])
             for _, row in R.iterrows()}
elif ("state" in R.columns) and ("action" in R.columns):
    r_sa = {(str(row["state"]), str(row["action"])): float(row["reward"])
            for _, row in R.iterrows()}
elif "state" in R.columns:
    r_s = {str(row["state"]): float(row["reward"]) for _, row in R.iterrows()}
else:
    raise ValueError("Rewards file must contain at least a 'state' column.")

def reward(s, a, sp):
    if r_sas is not None:
        return r_sas.get((s, a, sp), 0.0)
    if r_sa is not None:
        return r_sa.get((s, a), 0.0)
    return r_s.get(sp, r_s.get(s, 0.0))

group = {}
for (s, a), df in T.groupby([s_col, a_col]):
    group[(s, a)] = list(zip(df[sp_col].tolist(), df[p_col].tolist()))

gamma = 0.99
tol = 1e-10
max_iter = 200000

V = {s: 0.0 for s in states}

for _ in range(max_iter):
    delta = 0.0
    Vnew = {}

    for s in states:
        a = Pi_map.get(s, None)
        if a is None or (s, a) not in group:
            Vnew[s] = V[s]
            continue

        v = 0.0
        for sp, p in group[(s, a)]:
            v += p * (reward(s, a, sp) + gamma * V.get(sp, 0.0))

        Vnew[s] = v
        delta = max(delta, abs(v - V[s]))

    V = Vnew
    if delta < tol:
        break

expected_discomfort = {s: -V[s] for s in states}

Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [10]:
# YOUR CHANGES HERE

out = pd.DataFrame(
    {"state": list(expected_discomfort.keys()),
     "expected_discomfort": list(expected_discomfort.values())}
).sort_values("state")

out.to_csv("minimum-discomfort-values.tsv", sep="\t", index=False)

Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick (must have symptoms, exposed does not count) and 0 otherwise.
To be clear, the action does not matter for this reward function.


In [11]:
# YOUR CHANGES HERE

# Load original rewards to preserve format
R = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

R.columns = [c.strip().lower() for c in R.columns]

# Identify state column
if "state" not in R.columns:
    raise ValueError("Rewards file must contain a 'state' column.")

# Define symptomatic states
# Adjust this condition if your state naming differs
def is_sick(state):
    state = str(state).lower()
    return ("symptom" in state) or ("fever" in state) or ("sick" in state)

# Overwrite reward column
R["reward"] = R["state"].apply(lambda s: -1 if is_sick(s) else 0)

Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [12]:
# YOUR CHANGES HERE

R.to_csv("duration-rewards.tsv", sep="\t", index=False)

Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [13]:
# YOUR CHANGES HERE

T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
R = pd.read_csv("duration-rewards.tsv", sep="\t")

T.columns = [c.strip().lower() for c in T.columns]
R.columns = [c.strip().lower() for c in R.columns]

def pick(col_options, cols):
    for c in col_options:
        if c in cols:
            return c
    raise ValueError(f"Missing one of {col_options}; found columns={list(cols)}")

s_col  = pick(["state", "s"], T.columns)
a_col  = pick(["action", "a"], T.columns)
sp_col = pick(["next_state", "next", "sprime", "s_next", "sp"], T.columns)
p_col  = pick(["prob", "probability", "p"], T.columns)

T = T[[s_col, a_col, sp_col, p_col]].copy()
T[s_col] = T[s_col].astype(str)
T[a_col] = T[a_col].astype(str)
T[sp_col] = T[sp_col].astype(str)
T[p_col] = T[p_col].astype(float)

states = sorted(set(T[s_col]).union(set(T[sp_col])))

if "state" not in R.columns or "reward" not in R.columns:
    raise ValueError("duration-rewards.tsv must have columns 'state' and 'reward'.")

r_s = {str(row["state"]): float(row["reward"]) for _, row in R.iterrows()}

def reward_duration(s, a, sp):
    return r_s.get(s, r_s.get(sp, 0.0))

group = {}
actions_by_state = {}
for (s, a), df in T.groupby([s_col, a_col]):
    group[(s, a)] = list(zip(df[sp_col].tolist(), df[p_col].tolist()))
    actions_by_state.setdefault(s, set()).add(a)

gamma = 0.99
tol = 1e-10
max_iter = 200000

V = {s: 0.0 for s in states}

for _ in range(max_iter):
    delta = 0.0
    Vnew = {}

    for s in states:
        acts = actions_by_state.get(s, None)
        if not acts:
            Vnew[s] = V[s]
            continue

        best = -1e300
        for a in acts:
            v = 0.0
            for sp, p in group[(s, a)]:
                v += p * (reward_duration(s, a, sp) + gamma * V.get(sp, 0.0))
            if v > best:
                best = v

        Vnew[s] = best
        delta = max(delta, abs(best - V[s]))

    V = Vnew
    if delta < tol:
        break

policy = []
for s in states:
    acts = actions_by_state.get(s, None)
    if not acts:
        continue

    best_a = None
    best_q = -1e300
    for a in acts:
        q = 0.0
        for sp, p in group[(s, a)]:
            q += p * (reward_duration(s, a, sp) + gamma * V.get(sp, 0.0))
        if q > best_q:
            best_q = q
            best_a = a

    policy.append((s, best_a))

Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [14]:
# YOUR CHANGES HERE

out = pd.DataFrame(policy, columns=["state", "action"]).sort_values("state")
out.to_csv("minimum-duration-actions.tsv", sep="\t", index=False)

Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [15]:
# YOUR CHANGES HERE

T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
Rdur = pd.read_csv("duration-rewards.tsv", sep="\t")
Pi = pd.read_csv("minimum-duration-actions.tsv", sep="\t")

T.columns = [c.strip().lower() for c in T.columns]
Rdur.columns = [c.strip().lower() for c in Rdur.columns]
Pi.columns = [c.strip().lower() for c in Pi.columns]

def pick(col_options, cols):
    for c in col_options:
        if c in cols:
            return c
    raise ValueError(f"Missing one of {col_options}; found columns={list(cols)}")

s_col  = pick(["state", "s"], T.columns)
a_col  = pick(["action", "a"], T.columns)
sp_col = pick(["next_state", "next", "sprime", "s_next", "sp"], T.columns)
p_col  = pick(["prob", "probability", "p"], T.columns)

T = T[[s_col, a_col, sp_col, p_col]].copy()
T[s_col] = T[s_col].astype(str)
T[a_col] = T[a_col].astype(str)
T[sp_col] = T[sp_col].astype(str)
T[p_col] = T[p_col].astype(float)

states = sorted(set(T[s_col]).union(set(T[sp_col])))

Pi_map = {str(r["state"]): str(r["action"]) for _, r in Pi.iterrows()}

if "state" not in Rdur.columns or "reward" not in Rdur.columns:
    raise ValueError("duration-rewards.tsv must have columns 'state' and 'reward'.")

r_s = {str(row["state"]): float(row["reward"]) for _, row in Rdur.iterrows()}

cost = {s: (1.0 if r_s.get(s, 0.0) < 0 else 0.0) for s in states}

group = {}
for (s, a), df in T.groupby([s_col, a_col]):
    group[(s, a)] = list(zip(df[sp_col].tolist(), df[p_col].tolist()))

D = {s: 0.0 for s in states}
tol = 1e-10
max_iter = 200000

for _ in range(max_iter):
    delta = 0.0
    Dnew = {}

    for s in states:
        a = Pi_map.get(s, None)
        if a is None or (s, a) not in group:
            Dnew[s] = D[s]
            continue

        v = cost[s]
        for sp, p in group[(s, a)]:
            v += p * D.get(sp, 0.0)

        Dnew[s] = v
        delta = max(delta, abs(v - D[s]))

    D = Dnew
    if delta < tol:
        break

expected_sick_days = D

Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [16]:
# YOUR CHANGES HERE

out = pd.DataFrame(
    {"state": list(expected_sick_days.keys()),
     "expected_sick_days": list(expected_sick_days.values())}
).sort_values("state")

out.to_csv("minimum-duration-days.tsv", sep="\t", index=False)

Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [17]:
# YOUR CHANGES HERE

T = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
R = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

Pi_speed = pd.read_csv("minimum-duration-actions.tsv", sep="\t")
Pi_pamper = pd.read_csv("minimum-discomfort-actions.tsv", sep="\t")

T.columns = [c.strip().lower() for c in T.columns]
R.columns = [c.strip().lower() for c in R.columns]
Pi_speed.columns = [c.strip().lower() for c in Pi_speed.columns]
Pi_pamper.columns = [c.strip().lower() for c in Pi_pamper.columns]

def pick(col_options, cols):
    for c in col_options:
        if c in cols:
            return c
    raise ValueError(f"Missing one of {col_options}; found columns={list(cols)}")

s_col  = pick(["state", "s"], T.columns)
a_col  = pick(["action", "a"], T.columns)
sp_col = pick(["next_state", "next", "sprime", "s_next", "sp"], T.columns)
p_col  = pick(["prob", "probability", "p"], T.columns)

T = T[[s_col, a_col, sp_col, p_col]].copy()
T[s_col] = T[s_col].astype(str)
T[a_col] = T[a_col].astype(str)
T[sp_col] = T[sp_col].astype(str)
T[p_col] = T[p_col].astype(float)

states = sorted(set(T[s_col]).union(set(T[sp_col])))

Pi_speed_map = {str(r["state"]): str(r["action"]) for _, r in Pi_speed.iterrows()}
Pi_pamper_map = {str(r["state"]): str(r["action"]) for _, r in Pi_pamper.iterrows()}

if "reward" not in R.columns:
    raise ValueError(f"Rewards file must have a 'reward' column; found columns={list(R.columns)}")

if "next_state" not in R.columns:
    for alt in ["next", "sprime", "s_next", "sp"]:
        if alt in R.columns:
            R = R.rename(columns={alt: "next_state"})
            break

r_sas = None
r_sa = None
r_s = None

if ("state" in R.columns) and ("action" in R.columns) and ("next_state" in R.columns):
    r_sas = {(str(row["state"]), str(row["action"]), str(row["next_state"])): float(row["reward"])
             for _, row in R.iterrows()}
elif ("state" in R.columns) and ("action" in R.columns):
    r_sa = {(str(row["state"]), str(row["action"])): float(row["reward"])
            for _, row in R.iterrows()}
elif "state" in R.columns:
    r_s = {str(row["state"]): float(row["reward"]) for _, row in R.iterrows()}
else:
    raise ValueError("Rewards file must contain at least a 'state' column.")

def reward_discomfort(s, a, sp):
    if r_sas is not None:
        return r_sas.get((s, a, sp), 0.0)
    if r_sa is not None:
        return r_sa.get((s, a), 0.0)
    return r_s.get(sp, r_s.get(s, 0.0))

group = {}
for (s, a), df in T.groupby([s_col, a_col]):
    group[(s, a)] = list(zip(df[sp_col].tolist(), df[p_col].tolist()))

def evaluate_policy_discomfort(Pi_map, gamma=0.99, tol=1e-10, max_iter=200000):
    V = {s: 0.0 for s in states}
    for _ in range(max_iter):
        delta = 0.0
        Vnew = {}
        for s in states:
            a = Pi_map.get(s, None)
            if a is None or (s, a) not in group:
                Vnew[s] = V[s]
                continue
            v = 0.0
            for sp, p in group[(s, a)]:
                v += p * (reward_discomfort(s, a, sp) + gamma * V.get(sp, 0.0))
            Vnew[s] = v
            delta = max(delta, abs(v - V[s]))
        V = Vnew
        if delta < tol:
            break
    return {s: -V[s] for s in states}

speed_discomfort = evaluate_policy_discomfort(Pi_speed_map)
minimize_discomfort = evaluate_policy_discomfort(Pi_pamper_map)

Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [18]:
# YOUR CHANGES HERE

out = pd.DataFrame({
    "state": states,
    "speed_discomfort": [speed_discomfort[s] for s in states],
    "minimize_discomfort": [minimize_discomfort[s] for s in states],
})

out.to_csv("policy-comparison.tsv", sep="\t", index=False)

Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.